In [1]:
import os
import json
from datasets import Dataset
from pyprojroot import here
from transformers import AutoTokenizer
from langchain_core.prompts import PromptTemplate

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-405B-Instruct")

In [11]:
ANSWER_FORMATS = """- should be complete with the letter and correct answer from the list of given choices (Example answer:  K. Ananda))"""

In [3]:
REASONING_PROMPT = """Solve the given task by following the step-by-step reasoning plan in JSON filling in the values for the corresponding keys.
Phrase your final answer always as "The final answer is [answer]".

[answer] should be in one of the following formats:
{answer_formats}
    
Reasoning Structure:
{reasoning_structure}

Task:
{task_description}

Correctly follow the above JSON reasoning structure to solve the given task. Your response should be the filled JSON for the above reasoning structure."""


prompt = PromptTemplate.from_template(REASONING_PROMPT)
prompt

PromptTemplate(input_variables=['answer_formats', 'reasoning_structure', 'task_description'], input_types={}, partial_variables={}, template='Solve the given task by following the step-by-step reasoning plan in JSON filling in the values for the corresponding keys.\nPhrase your final answer always as "The final answer is [answer]".\n\n[answer] should be in one of the following formats:\n{answer_formats}\n    \nReasoning Structure:\n{reasoning_structure}\n\nTask:\n{task_description}\n\nCorrectly follow the above JSON reasoning structure to solve the given task. Your response should be the filled JSON for the above reasoning structure.')

In [5]:
base_path = str(here("evals/logs/self_discover/mistral"))
phase1 = os.path.join(base_path, "phaseI/t4d")
phase2 = os.path.join(base_path, "phaseII/t4d/t4d_eval")

base_path, phase1, phase2

('/home/ubuntu/dev/self-discover/evals/logs/self_discover/mistral',
 '/home/ubuntu/dev/self-discover/evals/logs/self_discover/mistral/phaseI/t4d',
 '/home/ubuntu/dev/self-discover/evals/logs/self_discover/mistral/phaseII/t4d/t4d_eval')

In [3]:
with open(os.path.join(phase1, "t4d-.txt"), "r") as f:
    reasoning_structure = f.read()

print(reasoning_structure)

```json
{
    "Identify the core task": {
        "Determine the main question or task that needs to be answered or solved:"
    },
    "Understand the context": {
        "Identify the key events and actions described in the observation that are relevant to the task:"
    },
    "Break down the observation": {
        "Segment the observation into smaller, sequential, or categorical parts to analyze it more effectively:"
    },
    "Identify relevant individuals": {
        "Determine who are the individuals directly involved in or affected by the task or plan mentioned in the observation:"
    },
    "Analyze perspectives": {
        "Consider the actions, plans, and preferences of each individual mentioned in the observation. Question any assumptions about their knowledge or intentions:"
    },
    "Define useful information": {
        "Determine what information would be beneficial for an individual to know to facilitate their plans or tasks:"
    },
    "Establish relevance": {
 

In [6]:
dataset = Dataset.load_from_disk(phase2)
dataset

Dataset({
    features: ['story', 'question', 'answer', 'characters', 'distracting_characters', 'self_discover_input', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 564
})

In [7]:
dataset[0]

{'story': 'Nathan entered the porch. Mia entered the porch. The radish is in the crate. Nathan exited the porch. Mia moved the radish to the drawer. Liam entered the front yard. Nathan and Mia plan to use the radish soon.',
 'question': 'Who among the individuals would benefit from receiving helpful information?\nA. Mia\nB. Nathan\nC. Liam\nD. None of the above',
 'answer': 'Nathan',
 'characters': ['Nathan', 'Mia', 'Liam'],
 'distracting_characters': ['Liam'],
 'self_discover_input': 'Observation:\nNathan entered the porch. Mia entered the porch. The radish is in the crate. Nathan exited the porch. Mia moved the radish to the drawer. Liam entered the front yard. Nathan and Mia plan to use the radish soon.\n\nQuestion:\nWho among the individuals would benefit from receiving helpful information?\nA. Mia\nB. Nathan\nC. Liam\nD. None of the above',
 'reasoning_structure': '`',
 'reasoning': '```json\n{\n    "Identify the core task": {\n        "Determine the main question or task that nee

In [15]:
len(tokenizer.encode(prompt.format(answer_formats=ANSWER_FORMATS, reasoning_structure=reasoning_structure, task_description=dataset[0]["self_discover_input"])))

519

In [16]:
input_token_count = 0
output_token_count = 0

for instance in dataset:
    input_token_count += len(tokenizer.encode(prompt.format(answer_formats=ANSWER_FORMATS, reasoning_structure=reasoning_structure, task_description=instance["self_discover_input"])))
    output_token_count += len(tokenizer.encode(instance["reasoning"]))

print(f"Input Token Count: {input_token_count}")
print(f"Output Token Count: {output_token_count}")

Input Token Count: 297525
Output Token Count: 374450


In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
dataset.push_to_hub("sachithgunasekara/self-discover-mistral-t4d-eval")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-t4d-eval/commit/b11314fbf8526e1ad7e463f1734093ed2b96043a', commit_message='Upload dataset', commit_description='', oid='b11314fbf8526e1ad7e463f1734093ed2b96043a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-t4d-eval', endpoint='https://huggingface.co', repo_type='dataset', repo_id='sachithgunasekara/self-discover-mistral-t4d-eval'), pr_revision=None, pr_num=None)